In [11]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import get_dates_datetime, get_dirname, temp_path, get_data_info, update_plate_info, \
get_current_folders, get_folders_by_plate_id


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.launching.run_super import run_parallel_transfer, find_state, directory_scratch, directory_project, directory_archive, run_parallel_transfer_to_archive
import dropbox
from amftrack.util.dbx import upload_folders
from subprocess import call
import logging
logging.basicConfig(stream=sys.stdout, level=logging.debug)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
directory_scratch

'/scratch-shared/amftrack/'

In [17]:
directory = directory_project
directory = directory_scratch

# all_folders_drop= get_current_folders("dropbox",True)

update_plate_info(directory,local=True)
all_folders_project = get_current_folders(directory,local=True)


analysed:   0%|          | 0/4 [00:00<?, ?it/s]

In [20]:
folders = all_folders_project

In [18]:
folders = all_folders_project.loc[all_folders_project["Plate"]=="94"]

In [11]:
directory_targ = os.path.join(directory_scratch,'temp')+'/'
# # directory_targ = os.path.join(directory_archive,'Prince_Data')+'/'
# directory_targ = '/projects/0/einf914/agg/'
# directory_targ = directory_project


**To transfer data from surfsara to dropbox**
- use the 'toward_drop.py' function including the folders of interest and using dir_drop as a parameter indicating where in dropbox the data should go


In [21]:
dir_drop = 'test'
delete = True
run_parallel_transfer('toward_drop.py',[dir_drop,delete],folders,5,'2:00:00','staging',cpus = 1,node = 'staging')

Submitted batch job 921149


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


**To transfer data from dropbox to surfsara**
- use the 'from_drop.py' function including the folders of interest
- unzip argument decides if folders must also be unzipped
- flatten argument decides if folders must be flatten to respect snellius paralell computing logic or kept in a folder for transfer to archive for example

In [9]:
dir_drop = 'prince_data'
unzip = True
flatten = True
run_parallel_transfer('from_drop.py',[directory_targ,dir_drop,unzip,flatten],folders,5,'50:00','staging',cpus = 1,node = 'staging')

Submitted batch job 639383


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.


In [34]:
from amftrack.transfer.functions.transfer import download, zip_file,unzip_file

unzip_file('/scratch-shared/amftrack/temp/20210901_2139_Plate34.zip','/projects/0/einf914/data/agg/20210901_2139_Plate34')

**To transfer data from dropbox to archive**
- use the 'from_drop.py' function including the folders of interest


In [4]:
folders = all_folders.loc[all_folders['Plate'].isin([792,94])]

In [48]:
len(folders)

1858

In [47]:
unzip = 'False'
flatten = False
dir_drop = 'prince_data'

directory_targ = os.path.join(directory_scratch,'temp')+'/'
directory = directory_targ
run_parallel_transfer('from_drop.py',[directory_targ,dir_drop,unzip,flatten],folders,5,'10:00','staging',cpus = 1,node = 'staging',name_job='archiving.sh')
run_parallel_transfer_to_archive(folders,directory,'2:00:00','staging',name_job='archiving.sh')

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 M

Submitted batch job 418130
Submitted batch job 418131
Submitted batch job 418132
Submitted batch job 418133


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 M

Submitted batch job 418134
Submitted batch job 418135
Submitted batch job 418136
Submitted batch job 418138


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 M

...

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 M

Submitted batch job 418535
Submitted batch job 418536
Submitted batch job 418537
Submitted batch job 418538


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 M

Submitted batch job 418539
Submitted batch job 418540
Submitted batch job 418541
Submitted batch job 418542
Submitted batch job 418543
Submitted batch job 418544
Submitted batch job 418545
Submitted batch job 418546


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 M

In [6]:
run_parallel_transfer_to_archive(folders,directory,'10:00','staging',name_job='archiving_no_wait.sh')

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 374733
Submitted batch job 374734
Submitted batch job 374735
Submitted batch job 374736
Submitted batch job 374737
Submitted batch job 374738
Submitted batch job 374739
Submitted batch job 374740


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 374741
Submitted batch job 374742
Submitted batch job 374743
Submitted batch job 374744


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 374745
Submitted batch job 374746
Submitted batch job 374747
Submitted batch job 374748
Submitted batch job 374749
Submitted batch job 374750
Submitted batch job 374751
Submitted batch job 374752


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [12]:
must_zip = True
must_unzip = False
run_parallel_transfer('within_surf.py',[directory,directory_targ,must_zip,must_unzip],folders,5,'10:00','staging',cpus = 1,node = 'staging')

Submitted batch job 242022


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [7]:
plates = set(folders['Plate'].values)

In [10]:
call(f"tar cvf /archive/cbisot/prince_data/94_20201123.tar {directory_targ}94_20201123*", shell=True)

tar: /archive/cbisot/prince_data/94_20201123.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


2

In [ ]:
def transfer_parallel_whole_plate(code, args, folders, num_parallel, time, name,cpus = 128,node = 'thin'):
    op_id = time_ns()
    folders.to_json(f'{temp_path}/{op_id}.json')# temporary file
    plates = set(folders['Plate'].values)
    length = len(plates)
    begin_skel = 0
    end_skel = length // num_parallel + 1
    args_str = [str(arg) for arg in args]
    arg_str = " ".join(args_str)
    arg_str_out = "_".join([str(arg) for arg in args if type(arg)!=str])
    for j in range(begin_skel, end_skel):
        start = num_parallel * j
        stop = num_parallel * j + num_parallel - 1
        ide = time_ns()
        my_file = open(path_job, "w")
        my_file.write(
            f"#!/bin/bash \n#Set job requirements \n#SBATCH --nodes=1 \n#SBATCH -t {time}\n #SBATCH --ntask=1 \n#SBATCH --cpus-per-task={cpus}\n#SBATCH -p {node} \n"
        )
        my_file.write(
            f'#SBATCH -o "{path_code}slurm/{name}_{arg_str_out}_{start}_{stop}_{ide}.out" \n'
        )
        my_file.write(f"source /home/cbisot/miniconda3/etc/profile.d/conda.sh\n")
        my_file.write(f"conda activate amftrack\n")
        my_file.write(f"for i in `seq {start} {stop}`; do\n")
        my_file.write(f"\t python {path_code}amftrack/pipeline/scripts/image_processing/{code} {arg_str} {op_id} $i &\n")
        my_file.write("done\n")
        my_file.write("wait\n")
        my_file.close()
        call(f"sbatch {path_job}", shell=True)

In [11]:
for file in listfiles:
    print (file.name,file.content_hash)

param.m c51bd427fc91a8869dd28bd0453ceac525009c1b9c7ee3bfae79278b539ec129
Analysis.zip ec46f97fea264eebc2fd9e2c608e71729aa4e61e6f34fe460b3bcb8d054d7d7d
Img.zip 6cc480fa0c5e53f6799b169aa711ad501b02f8d9c533b9ce78ba256ae0caa981
StitchedImage.tif ed1f3b02b099e6871735d6b450b9248e4d090b8a00eedeabdd33f0d8d96e66b8
PrinceMeasurement.csv dea1bb564af80efdf67c9912e6c2611e272bf1d3685217fb7ac7ebeb13fdc7a7


In [20]:
folders = folders.loc[folders['folder']=="20201124_1202_Plate39"]

In [37]:
from amftrack.util.dbx import upload_zip,upload,zip_file

In [51]:
from time import time_ns
def upload_zip(path_total,target,catch_exception=False):
    '''
    Handles upload of files and folders taking only a path as an argument. If the total path is a
    file it is uploaded directly. Otherwise the folder is zipped and uploaded. Upload of individual files is
    inneficient.
     '''
    if os.path.isdir(path_total):
        stamp = time_ns()
        path_zip = f'{temp_path}/{stamp}.zip'
        print(path_zip,target)
        zip_file(path_total, path_zip)
        upload(path_zip, target,
               chunk_size=256 * 1024 * 1024,catch_exception=catch_exception)
        # os.remove(path_zip)

    else:
        upload(path_total, target,
               chunk_size=256 * 1024 * 1024,catch_exception=catch_exception)

In [60]:
run_info['CrossDate'].str.replace("'","")

29    20201123
Name: CrossDate, dtype: object

In [61]:
from tqdm.autonotebook import tqdm
dir_drop = 'test'
run_info = folders.copy()
folder_list = list(run_info['folder'])
with tqdm(total=len(folder_list), desc="transferred") as pbar:
    for folder in folder_list:
        directory_name = folder
        run_info['unique_id'] = run_info['Plate'].astype(int).astype(str) + "_" + run_info['CrossDate'].str.replace("'","").astype(int).astype(str)
        line = run_info.loc[run_info['folder'] == directory_name]
        id_unique = line['unique_id'].iloc[0]

        path_snap = line['total_path'].iloc[0]
        for subfolder in os.listdir(path_snap):
            path_total = os.path.join(path_snap, subfolder)
            suffix = '.zip' if os.path.isdir(path_total) else ''
            target = f'/{dir_drop}/{id_unique}/{directory_name}/{subfolder}{suffix}'
            upload_zip(path_total,target)
        os.remove(path_info)
        pbar.update(1)
        break

transferred:   0%|          | 0/1 [00:00<?, ?it/s]

2022-05-13 17:21:51,534-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-13 17:21:51,789-[INFO]- dropbox:477 -> Request to files/upload
/scratch-shared/amftrack/temp/1652455313416874865.zip /test/94_20201123/20201124_1202_Plate39/Analysis.zip


zipping:   0%|          | 0/16 [00:00<?, ?it/s]

2022-05-13 17:22:03,594-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-13 17:22:03,852-[INFO]- dropbox:477 -> Request to files/upload
/scratch-shared/amftrack/temp/1652455327049142316.zip /test/94_20201123/20201124_1202_Plate39/Img.zip


zipping:   0%|          | 0/152 [00:00<?, ?it/s]

Uploaded:   0%|          | 0/835234945 [00:00<?, ?it/s]

2022-05-13 17:23:33,597-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-13 17:23:33,815-[INFO]- dropbox:477 -> Request to files/upload_session/start
2022-05-13 17:23:39,987-[INFO]- dropbox:477 -> Request to files/upload_session/append
2022-05-13 17:23:49,135-[INFO]- dropbox:477 -> Request to files/upload_session/append
2022-05-13 17:23:57,067-[INFO]- dropbox:477 -> Request to files/upload_session/finish
2022-05-13 17:24:00,149-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-13 17:24:00,362-[INFO]- dropbox:477 -> Request to files/upload
2022-05-13 17:24:01,906-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-13 17:24:02,141-[INFO]- dropbox:477 -> Request to files/upload


In [9]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
from amftrack.util.file import hash_file,compute_dropbox_hash

In [9]:
path_zip = '/scratch-shared/amftrack/temp/1652455327049142316.zip'
target = "/test/94_20201123/20201124_1202_Plate39/Img.zip"
# upload(path_zip, target,
#                chunk_size=256 * 1024 * 1024)

In [13]:

compute_dropbox_hash(path_zip),compute_dropbox_hash("/projects/0/einf914/data/20201124_1202_Plate39/param.m")

('af11dc71b6d98d0ad0a3974515ee6d6d998a88d3c60c9971ac79d2e3111683b0',
 'c51bd427fc91a8869dd28bd0453ceac525009c1b9c7ee3bfae79278b539ec129')

In [10]:
from amftrack.util.dbx import load_dbx
dir_drop="/test/94_'20201123'/20201124_1202_Plate39/param.m"
dbx = load_dbx()
response = dbx.files_get_metadata(dir_drop)
response.content_hash,compute_dropbox_hash("/projects/0/einf914/data/20201124_1202_Plate39/param.m")

2022-05-16 08:41:17,944-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-16 08:41:18,152-[INFO]- dropbox:477 -> Request to files/get_metadata


('c51bd427fc91a8869dd28bd0453ceac525009c1b9c7ee3bfae79278b539ec129',
 'c51bd427fc91a8869dd28bd0453ceac525009c1b9c7ee3bfae79278b539ec129')

'c51bd427fc91a8869dd28bd0453ceac525009c1b9c7ee3bfae79278b539ec129'

In [74]:
hashe = hash_file("/projects/0/einf914/data/20201124_1202_Plate39/param.m")
hashe,len(hashe)

('6cac77b01d2cc27c014129a2cae001c06925ea0bf7235703b9bfad792456d1ab', 64)